In [1]:
import tweepy
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

In [2]:
#Define token access
costumer_key = '3Fpt5yX0rRcx3HltSvOMLWcnN'
costumer_secret_key = 'WqPVb5ELVOORF2BFM059WqNJCYxXlHRKBcqaQ5Re05J6CV1ZVc'
access_token = '1393767320611442692-nLgQB8fzUWpqFRYqJcBurY8SGrnaCx'
access_token_secret = 'RsLQDfSDYHryndKlJ9CU7rI8hvE3Qlw5hpBhuoJbiqIdH'

In [3]:
#Twitter authentication 
auth = tweepy.OAuthHandler(costumer_key,costumer_secret_key)
auth.set_access_token(access_token,access_token_secret)
#Instance
api = tweepy.API(auth, wait_on_rate_limit_notify=True)

In [4]:
all_replies = []

In [7]:
def find_tweet_ids(name, iterations):
    global api
    tweet_ids = []
    for _ in range(iterations):
        cuenta = api.get_user(name)
        statuses = cuenta.timeline(count = 1000, tweet_mode='extended')
        for status in statuses:
            tweet_ids.append(str(status.id))
    return set(tweet_ids)

## Cuentas para comentarios
- @MarchaOrgulloPe
- @LgbtPeru

In [8]:
#First Account
lis_id2 = list(find_tweet_ids('@MarchaOrgulloPe', 10))

In [9]:
len(lis_id2)

200

In [12]:
def get_replies(cad):
    options =  webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-extensions')
    executable_path = r"C:\Users\Paola\AnaliticaWeb\chromedriver.exe"
    driver = webdriver.Chrome(executable_path = executable_path, chrome_options=options)
    driver.maximize_window()
    time.sleep(3)
    # Inicializamos el navegador en la url
    driver.get(cad)
    time.sleep(3)
    # conseguir altura de pagina
    last_position = driver.execute_script("return window.pageYOffset;")
    html = requests.get(cad).content

    comment_tweet=[]
    while True:
        html = driver.page_source
        soup = bs(html,'lxml')
        comments = soup.find_all('div', {'class': 'css-901oao r-1fmj7o5 r-1qd0xha r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0'})
        for comment in comments:
            comment_tweet.append(comment.text)
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(1)
        curr_position = driver.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            break
        else:
            last_position = curr_position     
    driver.quit()
    return comment_tweet

In [17]:
replies_ac1 = []
for id_ in tqdm(lis_id2):
    link = 'https://twitter.com/MarchaOrgulloPe/status/'+id_
    tweet_replies = get_replies(link)
    for texto in tweet_replies:
        fila = [ id_ , texto ]
        if fila not in replies_ac1:
            replies_ac1.append(fila)

  0%|                                                                                          | 0/200 [00:00<?, ?it/s]<ipython-input-12-3e9702b4a0f0>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path = executable_path, chrome_options=options)
100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [1:34:14<00:00, 28.27s/it]


In [19]:
comentarios_dataframe= pd.DataFrame(replies_ac1, columns=['TWEET_ID','COMENTARIO'])

In [20]:
comentarios_dataframe

,TWEET_ID,COMENTARIO
0,1405656713500450817,"HILO \nEn las elecciones del año 2016, la disc..."
1,1405656713500450817,No olvidamos el pacto que firmó la señora Keik...
2,1405656713500450817,"El Colectivo Marcha del Orgullo, organizador d..."
3,1400516588470681604,ORGULLO TAMBIÉN ES AUTOCUIDADO \nReafirmamos n...
4,1400516588470681604,"Así mismo, aclaramos que cualquier persona que..."
...,...,...
181,1395777899085668354,Por esto y mucho más marcharemos este sábado 2...
182,1395777899085668354,"Traer carteles, banderolas y banderas LGBTIQ+...."
183,1381021758308950021,Un voto con convicción jamás será un voto perdido
184,1381021758308950021,Votar por quienes tienen el compromiso de logr...


In [21]:
comentarios_dataframe.to_excel('Data/replies_account1.xlsx')